# Extraction and Retrieval with Docling and Langchain DocLoaders

In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
import os
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
from qdrant_client import QdrantClient
from IPython.display import display, Markdown


# Params

In [72]:
INDEX = "althera-openai-01"  # collections in quadrant
EMB_MODEL = "text-embedding-3-small"
EMB_DIM = 1536
RETRIEVE_K=3

# Environment Variables

In [73]:
load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']


# Clients

In [74]:

client_qdrant = QdrantClient(":memory:")

client_openai = OpenAI(
    api_key=OPENAI_API_KEY
)


In [75]:
path_data = Path() / "data"
path_input = path_data / "raw"

# Extraction

In [76]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType


FILE_PATH = [(path_input / "Divulgacion-Planetaria-Althera.pdf").as_posix()]  # for multiple files
EXPORT_TYPE = ExportType.MARKDOWN   # ExportType.DOC_CHUNKS
TOKENIZER_NAME ="cl100k_base"

loader = DoclingLoader(
    file_path=FILE_PATH,
    export_type=EXPORT_TYPE,
)

corpus_docling = loader.load()

c:\Users\manuelalberto.romero\Documents\repos\dslabs\mbit\peia-oct25\dslab-rag-retrieval\.venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
c:\Users\manuelalberto.romero\Documents\repos\dslabs\mbit\peia-oct25\dslab-rag-retrieval\.venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


In [77]:
type(corpus_docling)

list

In [78]:
# ExportType.DOC_CHUNKS -> 68 chunks
# ExportType.MARKDOWN -> 1 chunk
len(corpus_docling)

1

In [79]:
type(corpus_docling[0])

langchain_core.documents.base.Document

In [80]:
Markdown(corpus_docling[0].page_content[:500])

## Un nuevo y fascinante vecino: Althéra

## Índice

1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras

## 1. Historia del descubrimiento

## 1.1 Primeras observaciones y sospechas iniciales

El sistema binario Althéra ( HD 4579 AB ) fue detectado por primera vez en el 

In [81]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter_rcs = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name=TOKENIZER_NAME,
    chunk_size=256,  # from_tiktoken_encoder: tokens
    chunk_overlap=25  # tokens
)
corpus_rcs = text_splitter_rcs.split_documents(corpus_docling)  # corpus is a list of LG Documents
len(corpus_rcs)


51

# Index

In [82]:
from uuid import uuid4

lst_chunks = [doc.page_content for doc in corpus_rcs]

In [83]:
lst_embeddings = []

response = client_openai.embeddings.create(
    input=lst_chunks,
    model=EMB_MODEL
)

for vec in response.data:
    lst_embeddings.append(vec.embedding)


print(f"{len(lst_embeddings)}")
print(f"{len(lst_embeddings[0])}")  # embeddings dimension

51
1536


In [84]:
from qdrant_client import models, QdrantClient

In [85]:
from uuid import uuid4
from qdrant_client import models, QdrantClient
# src.embeddings. create_index_points() 
lst_qdrant_pts = []

for content, vector in zip(lst_chunks, lst_embeddings):
    qdrant_point = models.PointStruct(
        id=str(uuid4()),  # uuid aleatorio
        payload={
            "text": content,
            "source": "Divulgacion-Planetaria-Althera.pdf",
        },
        vector=vector  # se puede buscar directamente en la lista por indice del DF pivotado
    )
    lst_qdrant_pts.append(qdrant_point)

len(lst_qdrant_pts) == len(corpus_rcs)  # comprobar

True

In [86]:
from qdrant_client.http.models import Distance, VectorParams

if not client_qdrant.collection_exists(INDEX):
    print(f"Creating collection {INDEX}...")
    client_qdrant.create_collection(
        collection_name=INDEX,
        vectors_config=VectorParams(size=EMB_DIM, distance=Distance.COSINE),
    )
else:
    print(f"Collection {INDEX} already exists")

if client_qdrant.get_collection(INDEX).points_count ==0:
    print("Loading chunks on VDB...")
    client_qdrant.upsert(
        collection_name=INDEX,
        points=lst_qdrant_pts
    )
else:
    print("There are already some chunks on the VDB")



Creating collection althera-openai-01...
Loading chunks on VDB...


In [89]:
client_qdrant.get_collection(INDEX).points_count

51

In [90]:
Markdown(lst_chunks[3])

## 1.3 Descubrimiento revolucionario de la zona habitable circumbinaria

El hallazgo más impactante llegó en 2034, cuando la misión LUVOIR-B (Large UV/Optical/IR Surveyor) detectó la firma espectral de vapor de agua, oxígeno molecular y metano en la atmósfera de Aurelia III , un planeta ubicado en la zona habitable del sistema, orbitando a ambos soles. Este fue el primer caso documentado de un mundo potencialmente habitable en un sistema binario cercano -a tan solo 42,7 años luz de la Tierra -, lo que lo convierte en un candidato ideal para futuras misiones de exploración interestelar.

## 1.4 Importancia científica y proyección futura

El descubrimiento de Althéra revolucionó la astrobiología y la física orbital por tres razones clave:

# Retrieval

In [94]:
# Embbed the query
_q_text = "distancia de althera a la tierra"


response = client_openai.embeddings.create(
    input=[_q_text],
    model=EMB_MODEL
)
_q_vec = response.data[0].embedding


# Query 
resp = client_qdrant.query_points(
    collection_name=INDEX,
    query=_q_vec,  
    limit=RETRIEVE_K  # K 
)

print(f"Question: {_q_text}")
for point in resp.points:
    doc_retrieved = point.payload['text']
    print(f"{point.id=}")
    print(f"{point.score=}")
    print(f"Doc: {doc_retrieved[:1000]}...")
    print("-"*30)

Question: distancia de althera a la tierra
point.id='1af43445-4f9a-45e1-b9d1-a42b53ca777e'
point.score=0.5021021569477758
Doc: Como anexo al estudio de la arquitectura orbital de Althéra, resulta relevante considerar el límite teórico de estabilidad circumbinaria definido por Holman &amp; Wiegert (1999). Esta formulación permite calcular la distancia mínima a la que un planeta puede mantener una órbita estable alrededor de dos estrellas, en función de la separación, la excentricidad del binario y la relación de masas estelares. Aplicado a Althéra, el valor obtenido es de aproximadamente 1,19 UA , lo que implica que cualquier órbita interior a esa distancia sufriría perturbaciones gravitatorias capaces de desestabilizarla a largo plazo. Sin embargo, al incorporar factores adicionales como resonancias con los gigantes exteriores, excentricidades planetarias y variaciones inducidas por la actividad estelar, el margen práctico de seguridad se sitúa en torno a 1,8 UA . Esta cifra explica po